先匯入雲端硬碟

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


匯入所需模組

In [41]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

帶入VGG19的模型(去除模型尾端)並凍結訓練參數

In [3]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)
for layer in vgg19.layers:
    layer.trainable = False

建立模型尾端密集神經層

In [4]:
# 建立空的模型後, 把載入的 VGG19 模型的神經層加進去：
model = Sequential()
model.add(vgg19)

# 在 VGG19 神經層後面增加負責分類的新神經層：
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(20))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(2, activation='softmax', name='predictions'))

將圖片擴增

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

定義訓練與驗證資料的生成器 

In [6]:
# 設定批次量
batch_size=32

# 定義訓練與驗證資料的生成器 

train_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/Colab Notebooks/hotdog_or_not/train',  # directory 請視自己的範例存放路徑彈性修改
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=50)

valid_generator = valid_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/Colab Notebooks/hotdog_or_not/test',   # directory 請視自己的範例存放路徑彈性修改
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=50)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


開始訓練模型

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, steps_per_epoch=15, epochs=8, validation_data=valid_generator,validation_steps=15)


Epoch 1/8
15/15 [==============================] - 618s 42s/step - loss: 0.6753 - accuracy: 0.6202 - val_loss: 0.5480 - val_accuracy: 0.7583
Epoch 2/8
15/15 [==============================] - 617s 43s/step - loss: 0.4924 - accuracy: 0.7618 - val_loss: 0.4847 - val_accuracy: 0.7812
Epoch 3/8
15/15 [==============================] - 593s 41s/step - loss: 0.3924 - accuracy: 0.8219 - val_loss: 0.4658 - val_accuracy: 0.7812
Epoch 4/8
15/15 [==============================] - 616s 43s/step - loss: 0.3806 - accuracy: 0.8262 - val_loss: 0.4846 - val_accuracy: 0.7708
Epoch 5/8
15/15 [==============================] - 593s 41s/step - loss: 0.3285 - accuracy: 0.8605 - val_loss: 0.4786 - val_accuracy: 0.7750
Epoch 6/8
15/15 [==============================] - 601s 41s/step - loss: 0.3581 - accuracy: 0.8562 - val_loss: 0.4685 - val_accuracy: 0.7729
Epoch 7/8
15/15 [==============================] - 616s 43s/step - loss: 0.3154 - accuracy: 0.8691 - val_loss: 0.4562 - val_accuracy: 0.7833
Epoch 8/8
15/

儲存模型

In [8]:
model.save('model.h5')

讀取模型


In [4]:
model = load_model('/content/model.h5')

Instructions for updating:
Colocations handled automatically by placer.


讀取圖片

In [42]:
#readpic=cv2.imread("/content/drive/MyDrive/Colab Notebooks/138933.jpg")#這張圖片是符合熱狗堡的
# readpic=cv2.imread("/content/drive/MyDrive/Colab Notebooks/8119.jpg")#這張是NG的
readpic=cv2.imread("/content/drive/MyDrive/Colab Notebooks/whk066-088.jpg")#這張圖片是符合熱狗堡的


圖片變更尺寸到224X224

In [43]:
resize_pic=cv2.resize(readpic,(224,224))
resize_pic=resize_pic.reshape([-1,224,224,3])
print(resize_pic.shape)
resize_pic=resize_pic/255

(1, 224, 224, 3)


預測圖片

In [44]:
model.predict(resize_pic)

array([[0.6059855 , 0.39401448]], dtype=float32)